In [1]:
!pip install transformers datasets peft accelerate bitsandbytes torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 24.3 MB/s eta 0:00:00:00:0100:01


In [2]:
# Step 2: Import libraries
from datasets import Dataset
import json
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model
import torch
import os

In [3]:

import os
os.environ["HF_TOKEN"]=""

os.environ["WANDB_DISABLED"] = "true"  # Force-disable wandb

In [4]:
# Load the dataset:



# Step 3: Load and prepare dataset
with open("/kaggle/input/datawalm/walmart-qa.json") as f:
    data = json.load(f)

# Format as instruction-following data
formatted_data = [
    {"text": f"### Instruction: {d['question']}\n### Response: {d['answer']}"}
    for d in data
]

dataset = Dataset.from_list(formatted_data)
dataset = dataset.train_test_split(test_size=0.1)

In [5]:
# Step 4: Load tokenizer
model_id = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,  # Increased for better context
        padding="max_length"
    )
    # Add labels (shifted input_ids for causal LM)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [6]:
# Step 5: Configure 4-bit QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

lora_config = LoraConfig(
    r=16,  # Higher rank for better adaptation
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


In [7]:
# Step 6: Load model with QLoRA
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",  # Automatically uses GPU
    torch_dtype=torch.float16
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Should show ~0.1% parameters


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

trainable params: 3,407,872 || all params: 1,239,222,272 || trainable%: 0.2750


In [8]:
# Step 7: Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [9]:
# Step 8: Training arguments (GPU optimized)
training_args = TrainingArguments(
    output_dir="./walmart-llama3-results",
    per_device_train_batch_size=8,  # Higher batch size for GPU
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-4,  # Higher learning rate for QLoRA
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    report_to="none",
    remove_unused_columns=False,
    fp16=True,  # Mixed precision training
    optim="paged_adamw_8bit",  # Optimized for 8-bit training
    warmup_ratio=0.1,
    lr_scheduler_type="cosine"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
# Step 9: Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)


<ipython-input-10-8a801911bb95>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
# Step 10: Train and save
print("Starting training...")
trainer.train()

Starting training...


Epoch,Training Loss,Validation Loss
1,No log,2.418840


TrainOutput(global_step=3, training_loss=5.283638636271159, metrics={'train_runtime': 18.8105, 'train_samples_per_second': 3.509, 'train_steps_per_second': 0.159, 'total_flos': 113999005876224.0, 'train_loss': 5.283638636271159, 'epoch': 1.6666666666666665})

In [13]:
# Save adapter only (small file)
model.save_pretrained("walmart-llama3-adapter")


In [14]:
# Step 11: Inference
def generate_response(question):
    prompt = f"### Instruction: {question}\n### Response:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [15]:
# Test
print("\nGenerated response:")
print(generate_response("What is Walmart's price match policy?"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Generated response:
### Instruction: What is Walmart's price match policy?
### Response: Walmart does not offer a refund or credit for items that were returned because they did not fit the customer. The company states it will refund customers who have merchandise that was purchased at full retail and then found to be defective in material, workmanship or quality.
#### Explanation:
Walmart does not accept returns on items with physical defects even if the item has been used. A company spokesperson said it would not honor returns for items that are too small or too large. Items that do not meet size requirements may still be eligible for return under certain circumstances depending upon the nature of the defect. For example, an item that does not meet its intended use
